# Environments

En JOS a los <b>procesos</b> se les llama <b>environments</b>. Asi que cuando hagamos referencia a un <b>environment</b> estaremos hablando de un <b>proceso</b> y viceversa.

## Estructura de un Environment 

La siguiente es la estructura de un proceso. Como podemos ver tiene diferentes cosas como el <b>env_id</b>, <b>env_parent_id</b> osea el <b>id</b> el proceso y el de su padre. Ademas tiene una estructura para guardarse los registros que utiliza, esto es <b>env_tf</b>. Luego tambien tiene una variable que guarda su estado, esta es <b>env_status</b>. Luego tenemos <b>env_runs</b> que guarda la cantidad de veces que este proceso fue corrido. Y por ultimo <b>env_pgdir</b> que es el directorio de paginas que usa este proceso.

```c
struct Env {
	struct Trapframe env_tf;	// Saved registers
	struct Env *env_link;		// Next free Env
	envid_t env_id;			// Unique environment identifier
	envid_t env_parent_id;		// env_id of this env's parent
	enum EnvType env_type;		// Indicates special system environments
	unsigned env_status;		// Status of the environment
	uint32_t env_runs;		// Number of times environment has run

	// Address space
	pde_t *env_pgdir;		// Kernel virtual address of page dir
};
```

#### Estados posibles para un Environment

```c
enum {
	ENV_FREE = 0,
	ENV_DYING,
	ENV_RUNNABLE,
	ENV_RUNNING,
	ENV_NOT_RUNNABLE
};
```

## Almacenamiento de los Environments

La forma que JOS trabaja almacenando los procesos es creando un array estatico de 1024 posiciones de tamanio del struct del environment.

<img src="Imagenes/environments.jpg">

Este array JOS lo llama <b>envs</b>:

```c
struct Env *envs = NULL;
```

Al proximo libre lo llama <b>env_free_list</b>:

```c
static struct Env *env_free_list;
```

Al que esta actualmente corriendo lo llama <b>curenv</b>. Viene de <b>current environment</b>:

```c
struct Env *curenv = NULL;
```

## Creacion de los Environments

Para crear este array hacemos los siguiente:

```c
void mem_init_envs() {
	// Creo el array que contendra todos los environments
	envs = boot_alloc(NENV * sizeof(struct Env));
	// Inicializo el array
	memset(envs, 0, NENV * sizeof(struct Env));
}
```

<hr>

# ELF Binary (Executable & Linkable Format)

Los binarios elf son un tipo particular de binarios que cuentan con la siguiente estructura.

<img src="Imagenes/binary_elf_estructura.png" width="400px">

### Program Header Table

La <b>program header table</b> le dice al sistema como crear un <b>proceso imagen</b>. Esto arranca en un offset de este archivo que se llama <b>e_phoff</b>, esta palabra viene de <b>elf program header offset</b>, consiste en una tabla con una cantidad de entradas indicada en <b>e_phnum</b> esto es <b>elf program header number</b> cada una de tamanio <b>e_phentsize</b> esto es <b>elf program header entrie size</b>.

```c
#define ELF_MAGIC 0x464C457FU
```

```c
struct Elf {
	uint32_t e_magic; // must equal ELF_MAGIC
	uint8_t e_elf[12];
	uint16_t e_type;
	uint16_t e_machine;
	uint32_t e_version;
	uint32_t e_entry; 
	uint32_t e_phoff; // indica el comienzo de las entradas
	uint32_t e_shoff;
	uint32_t e_flags;
	uint16_t e_ehsize;
	uint16_t e_phentsize; // es el tamanio de una entrada
	uint16_t e_phnum; // indica la cantidad de entradas que tiene el binario
	uint16_t e_shentsize;
	uint16_t e_shnum;
	uint16_t e_shstrndx;
};
```

```c
struct Proghdr {
	uint32_t p_type;
	uint32_t p_offset;
	uint32_t p_va;
	uint32_t p_pa;
	uint32_t p_filesz;
	uint32_t p_memsz;
	uint32_t p_flags;
	uint32_t p_align;
};
```

<hr>

# CR3 (Control Register 3)

Este registro almacena la direccion de memoria donde arranca la tabla <b>page directory</b>

<hr>

# Alocar el environment

La funcion <b>env_alloc</b> agarra un <b>environment</b> de la lista de <b>free environments</b> y le rellena sus caracteristicas, el <b>env_id</b>, <b>env_parent_id</b>, <b>env_type</b>, <b>env_status</b>, <b>env_runs</b> y por ultimo los registros que es el <b>env_tf</b>.

Los registros que rellena son el <b>ds (data segment)</b>, <b>es</b>, <b>ss</b>, <b>esp (stack pointer)</b>. Y el que no rellenamos es el <b>eip (instruction pointer)</b> ese lo rellenamos despues, cuando hallamos cargado el programa.

```c
int
env_alloc(struct Env **newenv_store, envid_t parent_id)
{
	int32_t generation;
	int r;
	struct Env *e;

	if (!(e = env_free_list))
		return -E_NO_FREE_ENV;

	// Allocate and set up the page directory for this environment.
	if ((r = env_setup_vm(e)) < 0)
		return r;

	// Generate an env_id for this environment.
	generation = (e->env_id + (1 << ENVGENSHIFT)) & ~(NENV - 1);
	if (generation <= 0)  // Don't create a negative env_id.
		generation = 1 << ENVGENSHIFT;
	e->env_id = generation | (e - envs);

	// Set the basic status variables.
	e->env_parent_id = parent_id;
	e->env_type = ENV_TYPE_USER;
	e->env_status = ENV_RUNNABLE;
	e->env_runs = 0;

	// Clear out all the saved register state,
	// to prevent the register values
	// of a prior environment inhabiting this Env structure
	// from "leaking" into our new environment.
	memset(&e->env_tf, 0, sizeof(e->env_tf));

	// Set up appropriate initial values for the segment registers.
	// GD_UD is the user data segment selector in the GDT, and
	// GD_UT is the user text segment selector (see inc/memlayout.h).
	// The low 2 bits of each segment register contains the
	// Requestor Privilege Level (RPL); 3 means user mode.  When
	// we switch privilege levels, the hardware does various
	// checks involving the RPL and the Descriptor Privilege Level
	// (DPL) stored in the descriptors themselves.
	e->env_tf.tf_ds = GD_UD | 3;
	e->env_tf.tf_es = GD_UD | 3;
	e->env_tf.tf_ss = GD_UD | 3;
	e->env_tf.tf_esp = USTACKTOP;
	e->env_tf.tf_cs = GD_UT | 3;
	// You will set e->env_tf.tf_eip later.

	// commit the allocation
	env_free_list = e->env_link;
	*newenv_store = e;

	cprintf("[%08x] new env %08x\n", curenv ? curenv->env_id : 0, e->env_id);
	return 0;
}
```

# Crear el environment

La funcion <b>env_create</b> se llama asi porque <b>aloca</b> el environment usando la funcion <b>env_alloc</b> y luego carga el <b>binary elf</b> en el <b>environment</b> usando la funcion <b>load_icode</b>.

```c
void
env_create(uint8_t *binary, enum EnvType type)
{
	// nuevo environment
	struct Env* new_env;
	// alocamos el nuevo environment
	int err = env_alloc(&new_env, 0);
	if (err < 0) panic("env_create: %e", err);
	// le cargamos el codigo del binario
	load_icode(new_env, binary);
}
```

# Cargar y mapear el programa al environment

Esta funcion se encarga de cargar a memoria el <b>binario elf</b> y lo mapea a memoria. Una observacion importante es que cada environment tiene su propio <b>page directory</b>, esto significa que dependiendo el proceso que estemos corriendo vamos a tener que cambiar el page directory, y esto se logra rapidamente indicando donde esta el page directory del proceso en el registro <b>cr3</b>. En la siguiente funcion vemos que como necesitamos usar el page directory del environment, antes de setearlo en cr3 hacemos un backup del cr3 de kernel. 

Otra cosa que hace esta funcion es que setea el instruction pointer del proceso, es decir donde le precesador tiene que comenzar a leer instrucciones para ejecutar. Esta direccion esta indicada en <b>elf_bin->e_entry</b>.

```c
static void
load_icode(struct Env *e, uint8_t *binary)
{
	// antes de arrancar me guardo el anterior cr3
	// que contiene la direccion de memoria fisica  
	// donde arranca el directorio de paginas del proceso
	unsigned int previous_cr3 = rcr3();

	// seteo el cr3 con la direccion fisica de la tabla 
	// de paginas de este proceso
	lcr3(PADDR(e->env_pgdir));

	// binario Elf
	struct Elf* elf_bin = (struct Elf*) binary;
	// header del Binario Elf
	struct Proghdr *elf_header;

	// la primer entrada se encuentra en donde esta el binario mas un offset
	// binary + binary->e_phoff 
	elf_header = (struct Proghdr *) (binary + elf_bin->e_phoff);

	// iteramos en las entradas del binario elf
	for(

		unsigned int ph_i = 0;

		// iteramos mientras la entrada actual sea menor a la cantidad de entradas
		// osea menor que elf_bin->e_phnum
		ph_i < elf_bin->e_phnum;

		// avanzamos en una entrada
		++ph_i,

		// indicamos que la proxima entrada es la actual mas el size de una entrada
		// osea es elf_header + elf_bin->e_phentsize
		elf_header =
			(struct Proghdr *) ((char *) elf_header + elf_bin->e_phentsize)
	){

		// solo cargamos las entradas que son de tipo ELF_PROG_LOAD
		if (elf_header->p_type != ELF_PROG_LOAD) continue;

		// aloco memoria en donde indica la direccion virtual el tamanio indicado
		region_alloc(
			/* environment */ e,                                   
			/* va */          (void *) elf_header->p_va,       
			/* size */        (size_t) elf_header->p_memsz);   

		// copio el esta seccion del programa en la region alocada
		memcpy(
			(void *) elf_header->p_va,                             /* destino */
			(void *) (binary + elf_header->p_offset),              /* origen  */
		    elf_header->p_filesz);                                 /*  size   */

		
		// seteo .bss en 0
		memset(
			(void *) (elf_header->p_va + elf_header->p_filesz),    /*    str    */
		    0,                                                     /* character */ 
		    (size_t) (elf_header->p_memsz - elf_header->p_filesz));/*     n     */
	}

	// seteo el instruction pointer del proceso
	e->env_tf.tf_eip = (uintptr_t) elf_bin->e_entry;

	// a continuacion vamos a allocar el stack que usara este programa

	// agarramos una pagina libre para el stack
	struct PageInfo* page = page_alloc(ALLOC_ZERO);

	// mapeo esta pagina en la direccion: USTACKTOP - PGSIZE
	page_insert(
		e->env_pgdir,
		page,
		(void*) (USTACKTOP - PGSIZE),
		PTE_U | PTE_W);

	// restauro el anterior page directory
	lcr3(previous_cr3);
}
```

# Correr el environment

La funcion <b>env_run</b> finalmente es la que corre el <b>environment</b>. Primero se fija si hay alguno corriendo y le cambia saca el flag que esta <b>RUNNING</b> luego cambia el <b>current environment</b>, osea cambia el <b>curenv</b>. Luego incrementa en 1 <b>env runs</b> y por ultimo setea en <b>cr3</b> la direccion del <b>page directory</b> del proceso.

```c
void
env_run(struct Env *e)
{
	// si hay un environment corriendo
	if (curenv) {
		// le cambiamos el estado de RUNNING a RUNNABLE
		curenv->env_status = ENV_RUNNABLE;
	}

	// el nuevo current environment es 'e'
	curenv = e;
	// le asignamos el estado: RUNNING
	curenv->env_status = ENV_RUNNING;
	// incrementamos en 1 el contador de veces 
	// que se corrio este proceso 
	curenv->env_runs += 1;
	// seteamos en ctr3 la direccion del  
	// page directory del proceso
	lcr3((uint32_t)PADDR(curenv->env_pgdir));
	env_pop_tf(&(curenv->env_tf));
}
```